# Tus datos en la nube: ETLs con Python en el cloud

## Caso: Dataset para análisis correlación entre la valoración del bitcoin y valores NASDAQ

El caso al que pretendemos dar respuesta trata de generar un set de datos que permita, posteriormente, entrenar algún modelo estadístico para evaluar si un subconjunto de valores bursátiles de empresas tecnológicas cotizadas mantiene correlación con la valoración del Bitcoin (BTC).  

Este tipo de modelos toman como *input* un tablón de datos con la siguiente estructura:
- Cada columna será tomada como una variable predictora.
- Existe una columna identificada como variable objetivo con los datos de la variable a predecir para su entrenamiento, test y validación.
- Cada registro representa una unidad observacional.

### 1- Diseño del prototipo de producto

Partiendo de que el objetivo es proveer de un buen producto para su posterior análisis, el requisito principal que este impone es básicamente el formato. En este caso, una tabla con variables predictoras, variable objetivo y con un registro para cada instancia de la unidad observacional a tratar. Vamos a intentar mapear estos requisitos formales a nuestro caso:

**Unidad observacional**

Cada registro de nuestra tabla corresponderá a un día en el cual se tomará una muestra de la valoración de cada empresa en NASDAQ y el valor del BTC.

2021-10-01 | `... variables predictoras ...` | `variable objetivo`

Por lo tanto, la `columna índice` será el propio día en el que se tome la muestra.

**Variables predictoras**

En este caso, será información relacionada con la valoración de las empresas del NASDAQ del estudio durante el día del muestreo. Para cada empresa cotizada se quiere obtener:
- Valoración al cierre de la jornada.
- Diferencia entre apertura y cierre.
- Tendencia de la jornada.
- Rango del día.

**Variable objetivo**

La variable objetivo en este caso será el valor del Bitcoin en el momento de la consulta.

**Prototipo del tablón final** <a name="tablon_final"></a>

In [2]:
import pandas as pd
import numpy as np

dias = pd.date_range('2021-10-01', '2021-10-10', freq='d')

valor_1_cierre = np.random.uniform(low=10.0, high=2000.0, size=10)

valor_1_dif = np.random.uniform(low=10.0, high=2000.0, size=10)

valor_1_tend = np.random.choice(a=('-', '+'), p=(0.5,0.5), size=10)

valor_1_rango = np.random.uniform(low=10.0, high=2000.0, size=10)

valor_2_cierre = np.random.uniform(low=10.0, high=2000.0, size=10)

valor_2_dif = np.random.uniform(low=10.0, high=2000.0, size=10)

valor_2_tend = np.random.choice(a=('-', '+'), p=(0.5,0.5), size=10)

valor_2_rango = np.random.uniform(low=10.0, high=2000.0, size=10)

valor_btc = np.random.uniform(low=10.0, high=20000.0, size=10)


In [3]:
pd.DataFrame({'V1_cierre': valor_1_cierre,
              'V1_dif': valor_1_dif,
              'V1_tend': valor_1_tend,
              'V1_rango': valor_1_rango,
              'V2_cierre': valor_2_cierre,
              'V2_dif': valor_2_dif,
              'V2_tend': valor_2_tend,
              'V2_rango': valor_2_rango,
              'V_BTC': valor_btc
             }, index=dias)

,V1_cierre,V1_dif,V1_tend,V1_rango,V2_cierre,V2_dif,V2_tend,V2_rango,V_BTC
2021-10-01,1522.128578,1216.320398,+,132.096401,984.164972,1897.878565,+,1726.995821,1427.281207
2021-10-02,1953.242897,1048.640601,-,1339.014751,439.960033,1125.425953,+,704.346426,17740.311524
2021-10-03,1331.877868,1720.068208,-,1006.028988,420.915478,713.249992,+,21.287510,2926.351254
2021-10-04,1763.848050,1859.672770,-,138.329005,609.856009,1312.718932,+,1464.635777,7378.060236
2021-10-05,1043.863434,1350.009186,+,40.598438,1359.568174,1772.486945,-,1294.714572,3794.381129
2021-10-06,1299.377220,330.475656,+,271.921582,1630.648184,628.529014,-,877.338943,16548.865301
2021-10-07,1349.850283,1855.872016,-,1310.406510,839.003449,1559.602844,-,740.899182,2106.411685
2021-10-08,1837.601382,92.944221,+,1731.897684,1768.055596,136.893282,-,1564.053337,9033.536712
2021-10-09,1225.835859,1778.222356,+,1114.190089,636.260812,1205.039940,+,1358.229436,3731.607484
2021-10-10,765.067183,362.218321,-,1230.035325,1075.433884,584.252317,-,684.941044,9019.120644


### 2- Extracción

#### ¿De dónde obtenemos los datos?

Existen varias fuentes de datos disponibles para este tipo de valores. Sin embargo, hemos elegido los siguientes:

*Para los valores de las empresas a consultar en NASDAQ*

En este caso, una de las fuentes más usadas en Yahoo Finance. Para presentar una nueva tipología de fuentes de datos, presentaremos `yfinance`, una librería que hace de *wrapper* para Python de la API de Yahoo Finance.

Este tipo de librerías nos facilitan mucho el trabajar con APIs puesto que implementan con clases y métodos el tipo de interacciones que solemos hacer. Otro ejemplo de este tipo podría ser `tweepy`, que permite interaccionar con la API de Twitter.

*Para los valores de BTC*

Usaremos la API REST "directa" en este caso de coinbase.

#### ¿Con qué frecuencia obtenemos los datos?

Las valoraciones de BTC se van actualizando de forma continuada, sin embargo, NASDAQ tiene un calendario y horario para operar. En este caso, nos interesa obtener los datos de las jornadas en que se haya operado en NASDAQ una vez estas hayan finalizado (para poder obtener todas las variables que nos interesan).

*Fuente*: http://www.nasdaqtrader.com/trader.aspx?id=calendar

#### ¿Qué partición consultamos?

*Para los valores de las empresas a consultar en NASDAQ*

Vemos si `yfinance` permite:
- consultar datos para una empresa determinada
- obtener los datos solamente del último día de operación
- ver sobre qué variables nos devuelve los datos

https://pypi.org/project/yfinance/ - ver método `ticker.history()`

Vemos si podemos consultar para solo una empresa y el último día:

In [4]:
import yfinance as yf

tk = yf.Ticker('AMZN')
raw_df = pd.DataFrame(tk.history(period='1d'))

raw_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-07-12,112.160004,113.230003,108.339996,109.220001,54179100,0,0


La forma de obtener variables de interés es mediante subset.

In [5]:
raw_df[['Open', 'High', 'Low', 'Close']]

,Open,High,Low,Close
Date,,,,
2022-07-12,112.160004,113.230003,108.339996,109.220001


Concluimos lo siguiente:
- `yfinance` permite consultas por índice bursátil, por lo que vamos a trabajar con un dataframe por empresa como fuente de datos (lista de dataframes). Esto nos obliga a agregarlas para obtener el tablón que deseamos.
- la librería permite obtener una partición bastante próxima a nuestro producto deseado.
- Existen variables que habrá ingeniar a partir de las existentes.

*Para los valores de BTC*

El enfoque de la API de Coinbase es distinto, puesto que obtiene el valor del BTC en ese preciso momento. Por lo tanto, vamos a tener que consultarlo en un momento determinado y obtener su valor.

In [7]:
import requests

response = requests.get('https://api.coinbase.com/v2/prices/spot?currency=USD')
btc_raw = response.json()
btc_raw

{'data': {'base': 'BTC', 'currency': 'USD', 'amount': '19476.71'}}

In [12]:
btc_raw['data']['amount']

'19476.71'

### 3- Transformación

#### QA

- Los datos obtenidos desde yfinance nos garantiza un tipo de datos determinado puesto que es directamente una librería implementada en Python (ver documentación).

- De los que vienen a través de la API REST de coinbase habrá que asegurar el tipo de dato.

#### Ingeniería de características

Realizamos la ingeniería de variables previa al agregado de forma que el conjunto pueda programarse más eficientemente.

Como hemos visto en el prototipo, las variables que esperamos obtener son básicamente relacionadas con los índices bursátiles y el valor del BTC.

Entonces, habría que evalaur cuales de estas vienen directamente obtenidas de las fuentes de datos y cuales hay que ingeniar a partir de las otras. Vamos entonces a examinar qué variables pueden ser directamente obtenibles desde las fuentes de datos y qué variables hay que ingeniar. Para las que caigan en esa categoría, vamos a estudiar cómo generarlas.

*variables índices bursátiles*
- [indicebursatil]_cierre: obtenible directamente a través de la consulta a `yfinance`. Campo `Close`.
- [indicebursatil]_dif: hay que ingenierla desde el dataframe obtenido con `yfinance`: `Open` - `Close`.
- [indicebursatil]_tend: hay que ingeniarla. Podemos aprovechar el resultado de la variable superior y simplemente evaluar su símbolo.
- [indicebursatil]_rango: hay que ingeniarla desde el dataframe obtenido con `yfinance`: `High` - `Low`.
- valor_btc: obtenible directamente a través del fichero JSON de respuesta de la API de Coinbase.


#### Agregado

De la extracción e ingeniería de características, vamos a obtener lo siguiente:
- Un listado de dataframes con la información bursátil de cada índice.
- Un dataframe con el valor del BTC.

Para agregarlos, deberemos concatenar los dataframes. Si el valor de BTC podemos formatearlo a dataframe, nos será más fácil de llevarlo a cabo puesto que usaremos solo ese tipo y nos serviremos de métodos de Pandas.

Con todo esto, habremos generado nuestro tablón final.

### 4- Load

Sólamente nos quedará resolver dónde guardamos el tablón.

**Ubicación**

La aproximación que propongo en el caso es guardarlo en una base de datos cloud basada en SQL.

De tal forma tendremos los siguientes beneficios:
- Tendremos los datos de forma consistente por usar un RDBMS.
- El tamaño de nuestro data

Vamos a crear una base de datos (y por lo tanto, también una instancia) de Azure SQL Server (solución **PaaS**). Una vez desplegada, el flow va a guardar para cada día que obtenga datos, un registro nuevo con los valores de cada variable en una tabla determinada.

Cómo el objetivo de la base de datos será puramente para consultas en los posteriores posibles análisis, vamos a optar por un enfoque **desnormalizado**. De tal forma, solo crearemos una tabla que sea un reflejo del dataframe producto de la fase de análisis.

**Conexión**

Las bases de datos suelen implementar un protocolo mediante el cual permiten crear conexiones desde otros sistemas (comunmente llamadas cadenas de conexión) llamado ODBC (**O**pen **D**ata**b**ase **C**onnectivity).

La forma más común de implementar estas conexiones en Python es mediante la librería `pyodbc`, que permite, a través de una cadena de conexión, generar un cursor que ejecute queries SQL sobre la base de datos. Es el método óptimo para poder pasar a la base de datos solo los registros que genere cada dia (`INSERTS`) a la vez que usamos las menos librerías posibles (por lo tanto somos más eficientes con el uso de recursos).

Las interacciones con la base de datos que implementaremos en el flow serán los siguienteS:
- Creación de la tabla si no existe: de tal forma, podremos asegurar que si la tabla no existe sea creada y las inserciones se realicen de forma correcta.

- Inserción de los valores diarios: pasando como parámetros los valores del día.


Las tareas de las que se compondrá esta conexión son las siguientes:
- Validar la operación en mercados durante el día del flow. Descartar los registros del tablón en caso de que no se haya operado en NASDAQ ese día.
- Crear tabla si no existe.
- Validar si existe el registro. Si existe descartamos los registros.
- Inserción de los registros de la tabla.

### 5 - Resumen

Vamos a recapitular lo que hemos estudiado en este ejercicio de definición del flow.

- El producto final va a parecerse al prototipo que hemos definido [aquí](#tablon_final).

- Sin embargo, cada ejecución diaria del flow generará un tablón con una fila para ese día, que será insertada en la base de datos. De esta forma usaremos más eficientemente todos los recursos de nuestra infrastructura que intervienen.

- Vamos a ejecutar el flow diariamente cuando cierren los mercados consultados. Controlaremos que no se inserten datos en la base de datos cuando haya discordancia entre la fecha del último registro obtenido de los índices de NASDAQ con el de BTC.

- En la extracción, trabajaremos con fuentes de datos basadas en API REST y librerías propias (`yfinance`).

- En la transformación, llevaremos a cabo sobretodo tareas de ingeniería de variables y agregación de datos.

- Vamos a guardar los datos en una base de datos de Azure SQL Server que crearemos en el desarrollo del caso.